In [1]:
from genericpath import exists
import os
import inflect
import json
import logging

import nltk

try:
    nltk.find('corpora/wordnet')
except:
    nltk.download('wordnet')

try:
    nltk.find('corpora/stopwords')
except:
    nltk.download('stopwords')

import config as constants
from config import config

import numpy as np
import pandas as pd

from utils.loader.loader import ClaimDataLoader

from utils.helpers import format_and_regex
from utils.logging.helpers import log_and_warn, log_initialize

log_initialize(
    file_path=config["logging"]["log_path_claim_data"],
    file_mode=constants.LOG_FILE_MODE,
    log_level=constants.LOG_LEVEL,
    log_format_str=constants.LOG_FORMAT,
    days_keep=7
)

from utils.preprocess import WEIGHTS_PREPROCESS, WEIGHTS_WORD_PREPROCESS


C:\Users\Florian\Anaconda3\envs\data-x\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [12]:
import re

In [29]:
weights_db_file = config["data"]["weights_db"]
weights_db = loader.preprocess_weights_db(weights_db_file=weights_db_file, filename=config["data"]["weights_preprocessed_db"])

claim_data = pd.read_csv('data/output/revised/single_year/Pentatonic_Xact_Categories_5yr_categorized.csv')

In [68]:
chunk_size = 3
data = claim_data.copy()

item_desc = data["item_description"].unique().tolist()
category_desc = data["subcategory_prev"].unique().tolist()
item_desc = [item for item in item_desc if item == item]
category_desc = [category for category in category_desc if category == category]
weights = weights_db
weight_descriptions = weights_db[["primary_desc", "secondary_desc", "material", "dimensions", 
                                  "values_desc", "unit"]].iloc[613:617].to_dict('records')
           
for desc in weight_descriptions:
    matched_mask = True
    matched_item_desc = item_desc.copy() #all item descriptions in here from claims in the beginning
    matched_category_desc = category_desc.copy() # all item descriptions in here from claims in the beginning

    for key in desc: #

        if desc[key]:

            #logging.info("Processing {} '{}'...".format(key, desc[key]))
            
            if key != 'unit':

                if ";" not in desc[key]:
                    desc_split = [" ".join(desc[key].split()[i:i+chunk_size]) for i in range(0, len(desc[key].split()), chunk_size)]

                    if len(desc[key].split()) > 3:
                        logging.info("Length of '{}' is too long for regex search. Splitting in chunks".format(desc[key]))
                else:
                    desc_split = [desc[key]]

                for desc_chunk in desc_split:     

                    #logging.info("Processing chunk '{}' from {} '{}'".format(desc_chunk, key, desc[key]))

                    compiled_regex_desc = re.compile(format_and_regex(desc_chunk.lower(), permutations=True, is_synonyms=True))

                    #logging.info("Regex search: {}".format(compiled_regex_desc))

                    matched_item_desc = list(filter(compiled_regex_desc.match, matched_item_desc))
                    matched_category_desc = list(filter(compiled_regex_desc.match, matched_category_desc))

                    #florian - test output
                    #log_and_warn('testing the output here:{} / {}'.format(matched_item_desc, matched_category_desc))

                    if len(matched_item_desc) == 0 and len(matched_category_desc) == 0:
                        break

                matched_mask &= (weights[key] == desc[key])
            
                #print('--')
                #print(key)
                #print(matched_item_desc)
                #print('--')
                
            if key == 'unit':
                cleaned_matched_items = []
                #print('claims data:')
                #print(matched_item_desc)
                print('length of the weights match')
                print(len(weights[matched_mask]))
                unit_weights = weights[matched_mask].iloc[0]['unit']
                for el in matched_item_desc:
                    unit_claims = data[data.item_description == el].iloc[0]['item_unit_cd']
                    print(unit_weights)
                    if unit_claims == unit_weights:
                        cleaned_matched_items.append(el)
                #print('matched with:')
                #print(weights[matched_mask])
                #print(weights[matched_mask]['unit'])
                #print('----')
                #print(desc)
                #print(len(cleaned_matched_items))
                #print(len(matched_item_desc))
            
    #print('CHECK OUT THE MASK')
    #print(matched_mask)
    print('---')

length of the weights match
1
EA
---
length of the weights match
1
---
length of the weights match
5
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
EA
---
length of the weights match
1
EA
EA
EA
---


In [62]:
len(cleaned_matched_items)

len(matched_item_desc)

0

In [51]:
type(data[data.item_description == el].iloc[0]['item_unit_cd'])

str

In [44]:
for el in matched_item_desc:
    print(data[data.item_description == el].iloc[0]['item_unit_cd'])

SF
SF
SF
SF
SF
SF
SF
SF
SF
SF
SF
SF
SF
SF


In [43]:
data[data.item_description == 'COUNTERTOP - FLAT LAID PLASTIC LAMINATE'].iloc[0]['item_unit_cd']

'LF'